In [9]:
#importando las librerias
import nltk
nltk.download('punkt')
import numpy
import tensorflow
import random
import json

import pickle #guardar el modelo y no estar cargando a cada momento
from nltk.stem.lancaster import LancasterStemmer #transformar palabras
stemmer = LancasterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
#abriendo los datos e imprimiendo los datos
with open("/content/db.json", encoding='utf-8') as archivo:
    datos = json.load(archivo)
print(datos)

{'contenido': [{'tag': 'saludo', 'patrones': ['hola', 'buenos dias', 'necesito ayuda'], 'respuestas': ['hola en que puedo ayudarle', 'buenos dias en que puedo ayudarle', 'Hola buenos dias, necesita ayuda']}, {'tag': 'examengeneral', 'patrones': ['cuando sera el examen general?', 'examen general', 'cuando es examen general'], 'respuestas': ['el examen general sera el 28 de setiembre.', 'el examen general sera el 28 de setiembre.', 'el examen general sera el 28 de setiembre.']}, {'tag': 'examencepre', 'patrones': ['cuando sera el examen cepreuna?', 'examen cepreuna', 'cuando es examen cepreuna'], 'respuestas': ['el examen cepreuna sera el 2 de octubre.mi estimado', 'el examen cepreuna sera el 2 de octubre.', 'el examen cepreuna sera el 2 de octubre. postulante']}, {'tag': 'carreras', 'patrones': ['cuantas carreras hay?', 'cuantas carreras?', 'cuantas carreras hay en la universidad'], 'respuestas': ['mi estimado(a) hay un total de 33 carreras', 'mi estimado(a) hay un total de 33 carreras 

Separando las palabras

In [12]:
palabras = []
tags = []
datos_palabras = []
datos_contenido = []
for contenido in datos["contenido"]:
    for patrones in contenido["patrones"]:
        palabraux = nltk.word_tokenize(patrones) #toma una frase la separa el palabras
        palabras.extend(palabraux)
        datos_palabras.append(palabraux)
        datos_contenido.append(contenido["tag"])
        if contenido["tag"] not in tags:
          tags.append(contenido["tag"])
print(palabras)
print(datos_palabras)
print(datos_contenido)
print(tags) # palabras repetidas
#nos ayudara a encontrar las  palabras clave

['hola', 'buenos', 'dias', 'necesito', 'ayuda', 'cuando', 'sera', 'el', 'examen', 'general', '?', 'examen', 'general', 'cuando', 'es', 'examen', 'general', 'cuando', 'sera', 'el', 'examen', 'cepreuna', '?', 'examen', 'cepreuna', 'cuando', 'es', 'examen', 'cepreuna', 'cuantas', 'carreras', 'hay', '?', 'cuantas', 'carreras', '?', 'cuantas', 'carreras', 'hay', 'en', 'la', 'universidad', 'quien', 'es', 'el', 'rector', '?', 'quien', 'es', 'el', 'rector', 'quien', 'es', 'rector', 'de', 'la', 'universidad', 'donde', 'puedo', 'inscribirme', '?', 'donde', 'se', 'inscribe', '?', 'donde', 'se', 'puede', 'inscribirse', '?', 'como', 'puedo', 'inscribirme', '?', 'como', 'se', 'podra', 'inscribir', '?', 'como', 'se', 'puede', 'inscribirse', '?', 'adios', 'me', 'retiro', 'me', 'voy']
[['hola'], ['buenos', 'dias'], ['necesito', 'ayuda'], ['cuando', 'sera', 'el', 'examen', 'general', '?'], ['examen', 'general'], ['cuando', 'es', 'examen', 'general'], ['cuando', 'sera', 'el', 'examen', 'cepreuna', '?'], 

In [14]:
#pasamos palabras en minuscula recoremos toda la lista de palabras
palabras = [stemmer.stem(w.lower()) for w in palabras if w!= "?" ]
palabras = sorted(list(set(palabras)))
tags = sorted(tags)

entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]
#x = indice, documento = palabra
for x, documento in enumerate(datos_palabras):
    num=[]
    palabraux = [stemmer.stem(w.lower()) for w in documento]
    for w in palabras:
       if w in palabraux:
          num.append(1)
       else:
          num.append(0)
    filaSalida = salidaVacia[:]
    filaSalida[tags.index(datos_contenido[x])] = 1
    entrenamiento.append(num)
    salida.append(filaSalida)
print(entrenamiento)
print(salida)
#1 palabra encontrada que se va a necesitar el entremanieto

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

creando la red neuronal

In [5]:
pip install tflearn

     |████████████████████████████████| 112kB 4.0MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=654f1c888366dd565e438c13b8a3f44277158a286a92a4f1bb0f7cea53a5f3fe
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [19]:
import tflearn
from tensorflow.python.framework import ops
ops.reset_default_graph()

entrenamiento = numpy.array(entrenamiento)
salida = numpy.array(salida)

#tener espacio en blanco
ops.reset_default_graph()

#longitud de la una sola fila
red = tflearn.input_data(shape=[None, len(entrenamiento[0])])
red = tflearn.fully_connected(red, 24) #24 neuronas ocultas
red = tflearn.fully_connected(red, 24) 
red = tflearn.fully_connected(red, len(salida[0]), activation="softmax")
red = tflearn.regression(red) #que tan efectivo es...


modelo = tflearn.DNN(red)
modelo.fit(entrenamiento, salida, n_epoch=400, batch_size=48, show_metric= True) #la cantidad de mismos datos , cantidad de entradas de datos
modelo.save("modelo.tflearn")


---------------------------------
Run id: U5R9EX
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 24
Validation samples: 0
--
Training Step: 1  | time: 0.081s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 24/24
--
Training Step: 2  | total loss: 1.87150 | time: 0.007s
| Adam | epoch: 002 | loss: 1.87150 - acc: 0.0375 -- iter: 24/24
--
Training Step: 3  | total loss: 2.04150 | time: 0.007s
| Adam | epoch: 003 | loss: 2.04150 - acc: 0.2455 -- iter: 24/24
--
Training Step: 4  | total loss: 2.06971 | time: 0.005s
| Adam | epoch: 004 | loss: 2.06971 - acc: 0.4051 -- iter: 24/24
--
Training Step: 5  | total loss: 2.07610 | time: 0.005s
| Adam | epoch: 005 | loss: 2.07610 - acc: 0.2977 -- iter: 24/24
--
Training Step: 6  | total loss: 2.07780 | time: 0.007s
| Adam | epoch: 006 | loss: 2.07780 - acc: 0.2938 -- iter: 24/24
--
Training Step: 7  | total loss: 2

In [ ]:
def mainBot():
  while True:
    entrada = input("Tu: ")
    num = [0 for _ in range(len(palabras))]
    entradaProcesada = nltk.word_tokenize(entrada)
    entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]
    for palabraIndividual in entradaProcesada:
      #indice , elemento
        for i,palabra in enumerate(palabras):
            if palabra == palabraIndividual:
                num[i] = 1 #se ocupa la palabra
    resultados = modelo.predict([numpy.array(num)]) #probabilidad que que respuesta corresponde
    resultadosIndices = numpy.argmax(resultados)
    tag = tags[resultadosIndices]

    for tagAux in datos["contenido"]:
      if  tagAux["tag"] == tag:
          respuesta = tagAux["respuestas"]
    #  else: 
    #      respuesta = ["error"]
    #      exit()

    print("BOT: ", random.choice(respuesta))
    
    #print(resultados)
mainBot()    

Tu: hola
BOT:  Hola buenos dias, necesita ayuda
Tu: cuando es el examen general?
BOT:  el examen general sera el 28 de setiembre.
Tu: cuando es el examen cepreuna?
BOT:  el examen cepreuna sera el 2 de octubre. postulante
Tu: como puedo inscribirme?
BOT:  Para obtener mas informacion ingrese al siguiente link: https://admision.unap.edu.pe/admision
Tu: adios
BOT:  adios mi estimado(a)
